# Imports

In [53]:
import sys

modules_to_reload = [
    "src.utils.method_loggers",
    "src.utils.method_runners",
    "src.utils.metrics_calculators",
    "src.utils.tensor_handlers",
    "src.utils.trackers",
    "src.utils.video_controller",
]

for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

import gc
import os
from itertools import product

import numpy as np
import t3f

# import tensorflow as tf
import tensorly as tl
import torch
from dotenv import load_dotenv
from tqdm import tqdm

from src.utils.method_loggers import MethodLogger
from src.utils.method_runners import MethodRunner
from src.utils.save_frames import SaveFramesFactory
from src.utils.tensor_handlers import normalize_frames
from src.utils.trackers import GPUTorchMemoryTracker, RAMMemoryTracker, TimeTracker
from src.utils.video_controller import download_youtube_video, extract_frames

load_dotenv()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [32]:
torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch_device

device(type='cuda')

In [33]:
torch.cuda.empty_cache()

In [34]:
# tf_device = tf.config.list_physical_devices("GPU")[0].name
# tf_device

## Some params

In [60]:
video_url = "https://www.youtube.com/watch?v=eSKe2Vx-rpY"
proxy_url = os.getenv("PROXY_URL")
cache_dir = "../.cache/youtube"

log_file_path = "../.cache/method_logs.json"

method_logs_list = []

## Check how functions work

In [36]:
video_path = download_youtube_video(video_url, cache_dir=cache_dir.__str__(), proxy_url=proxy_url)

Видео уже загружено и закешировано: ../.cache/youtube/eSKe2Vx-rpY.mp4


In [37]:
video_frames, original_fps, frame_size = extract_frames(video_path)

In [38]:
video_frames.shape

(440, 360, 202, 3)

In [39]:
# processed_video_frames = process_frames(video_frames)

## Check original video

In [40]:
# show_frames_as_video(processed_video_frames)

# Implementations of Decompositions methods

Some packages which can decompose some dense type of tensors, from [this](https://arxiv.org/pdf/2103.13756) paper


Decomposition methods which used:
1. Canonical Polyadic Decomposition as PARAllel FACtors analysis (aka PARAFAC aka CPD aka CP)
2. Tucker Decomposition
3. Tensor Train
4. some variants of its (Other)

Tensor types:
1. Dense (D)
2. Sparse (S)
3. BlockSparse (BS)
4. Symmetric
5. Supersymmetric

Target system:
1. CPU (C)
2. GPU(G)
3. Distributed Memory (D)



| Method name                                                                             | Decomposition methods implemented | Tensor Type | Platform | Language         | Git | PyPI | Want to check | Checked     |
|-----------------------------------------------------------------------------------------|-----------------------------------|------------|----------|------------------|------|------|--------------|-------------|
| [AdaTM](https://github.com/hpcgarage/AdaTM)                                             | CP                                | S          | C        | C                | +    | ?    |              |             |
| [BTAS](https://github.com/ValeevGroup/BTAS)                                             | CP, Tucker                        | nan        | C        | C++              | +    | ?    |              |             |
| [CP-CALS](https://github.com/HPAC/CP-CALS)                                              | CP, Other                         | D          | C, G     | C++, Mat         | +    |      | +            |             |
| [CSTF](https://github.com/ZacBlanco/cstf)                                               | Other                             | S          | D        | Scala            | +    | ?    |              |             |
| [D-Tucker](https://datalab.snu.ac.kr/dtucker/resources/DTucker-v1.0.tar.gz)             | Tucker, Other                     | D          | C        | Matlab           |      | ?    |              |             |
| [DFacTo](http://www.joonheechoi.com/research.)                                          | CP                                | S          | C, D     | C++              |      | ?    |              |             |
| [EXATN](https://github.com/ORNL-QCI/exatn)                                              | TensorTrain                       | D          | C, D, G  | C++, Py          | +    |      | +            |             |
| [Genten](https://gitlab.com/tensors/genten)                                             | CP                                | D, S       | C, G     | C++              | +    |      | +            |             |
| GigaTensor                                                                              | CP                                | D          | C        | C++, Python      |      | ?    |              |             |
| [ITensor](https://github.com/ITensor/ITensor)                                           | TensorTrain                       | D, BS      | C, G     | C++, Julia       | +    |      | +            |             |
| [multiway](https://cran.r-project.org/web/packages/multiway/index.html)                 | CP, Tucker, Other                 | D          | C        | R                |      | ?    |              |             |
| [N-way toolbox](http://www.models.life.ku.dk/nwaytoolbox/download)                      | CP, Tucker, Other                 | D          | C        | Matlab           |      | ?    |              |             |
| [ParCube](https://www.cs.ucr.edu/~epapalex/src/parCube.zip)                             | CP                                | S          | C        | Matlab           |      | ?    |              |             |
| [ParTensor](https://github.com/neurocom/partensor-toolbox)                              | CP                                | D          | C, G     | C++              | +    |      | +            |             |
| [ParTI!](https://github.com/hpcgarage/ParTI)                                            | CP, Tucker                        | S          | C, G     | C, CUDA, Mat     | +    | ?    |              |             |
| [PLANC](https://github.com/ramkikannan/planc)                                           | CP                                | S          | C, D     | C++              | +    | ?    |              |             |
| [PLS toolbox](https://eigenvector.com/software/pls-toolbox/)                            | CP          , Tucker              | D          | C        | Matlab           |      | ?    |              |             |
| [Pytensor](https://code.google.com/archive/p/pytensor/source/default/source)            | Tucker                            | D, S       | C        | Python           |      | ?    |              |             |
| [rTensor](https://github.com/jamesyili/rTensor)                                         | CP, Tucker, Other                 | D          | C        | R                | +    |      | +            |             |
| [rTensor (randomized)](https://github.com/erichson/rTensor)                             | CP                                | D          | C        | Python           | +    |      | +       +    |             |
| [scikit-tensor](https://github.com/mnick/scikit-tensor)                                 | CP, Tucker, Other                 | D, S       | C        | Python           | +    | +    | +    +   +   | too old     |
| [Scikit-TT](https://github.com/PGelss/scikit_tt)                                        | TensorTrain                       | D          | C        | Python           | +    |      |     +   +    |             |
| [SPALS](https://github.com/dehuacheng/SpAls)                                            | CP                                | S          | C        | C++              | +    | ?    |              |             |
| [SPARTan](https://github.com/kperros/SPARTan)                                           | Other                             | S          | C        | Matlab           | +    | ?    |              |             |
| [SPLATT](https://github.com/ShadenSmith/splatt)                                         | CP                                | S          | C, D     | C, C++, Oct, Mat | +    | ?    |              |             |
| [SuSMoST](https://susmost.com/downloads.html)                                           | TensorTrain, Other                | D          | C        | Python           |      | ?    |              |             |
| [T3F](https://github.com/Bihaqo/t3f)                                                    | TensorTrain                       | D          | C, G     | Python           | +    | +    | +    + +     | in progress |
| [TDALAB](https://github.com/andrewssobral/TDALAB)                                       | CP                                | D, S       | C        | Python, Matlab   | +    |      | +         +  |             |
| [TeNPy](https://github.com/tenpy/tenpy)                                        | TensorTrain                       | D          | C        | Python           | +    | +    | +      + +   | in progress |
| [Tensor Fox](https://github.com/felipebottega/Tensor-Fox)                               | CP                                | D, S       | C        | Python, Matlab   | +    | +    | +    + +     | ?           |
| [Tensor package](http://www.gipsa-lab.fr/~pierre.comon/TensorPackage/tensorPackage.html) | CP                                | D          | C        | Matlab           |      | ?    |              |             |
| [Tensor Toolbox](https://gitlab.com/tensors/tensor_toolbox)                             | CP, Tucker, Other                 | D, S       | C        | Matlab           | +    |      | +            |             |
| [tensor_decomposition](https://github.com/cyclops-community/tensor_decomposition)       | CP, Tucker                        | D          | C, D     | Python           | +    |      | +        +   |             |
| [TensorBox](https://github.com/phananhhuy/TensorBox)                                    | CP, Tucker, Other                 | D, S       | C        | Matlab           | +    |      | +            |             |
| [TensorD](https://github.com/Large-Scale-Tensor-Decomposition/tensorD)                  | CP, Tucker                        | D          | C, G     | Python           | ?    | ?    |              |             |
| [TensorLab](https://www.tensorlab.net)                                                  | CP, Tucker, Other                 | D, S       | C        | Matlab           |      | ?    |              |             |
| [TensorLab+](https://www.tensorlabplus.net)                                             | CP, Other                         | D, S       | C        | Matlab           |      | ?    |              |             |
| [TensorLy](https://github.com/tensorly/tensorly)                                        | CP, Tucker, TensorTrain, Other    | D          | C, G     | Python           | +    | +    | +       + +  | in progress |
| [Three-Way](https://github.com/cran/ThreeWay)                                           | CP, Tucker                        | D          | C        | R                | +    |      | +            |             |
| [TNR](https://github.com/ycyuustc/matlab)                                               | Other                             | D          | C        | Matlab           | +    |      | +            |             |
| [TT-Toolbox](https://github.com/oseledets/TT-Toolbox)                                   | TensorTrain                       | D          | C, D, G  | Matlab, Python   | +    |      | +       +    |             |
| [xerus](https://git.hemio.de/xerus/xerus/)                                              | TensorTrain                       | D, S       | C        | C++              | +    |      | +            |             |

## TensorLy

In [41]:
# {‘numpy’, ‘mxnet’, ‘pytorch’, ‘tensorflow’, ‘cupy’}
# backend variants for tensorly
# tl.set_backend('pytorch')
# with tl.backend_context(‘pytorch’): ... pass

# video_frames_cuda = tl.tensor(video_frames.copy()).to(device)
# video_frames_cuda = tl.tensor(video_frames.copy())

### Params

In [42]:
# video_frames

### Tucker (tl.decomposition.tucker)

#### Params

In [43]:
tl.SVD_FUNS

['truncated_svd', 'symeig_svd', 'randomized_svd']

In [44]:
tensor_rank_mapping = {
    "video": (video_frames.shape[0] // 2, video_frames.shape[1], video_frames.shape[2], video_frames.shape[3]),
}

input_tensors = {
    "video": video_frames.copy(),
}

n_iter_max_param = 100

svd_params = ["truncated_svd", "symeig_svd", "randomized_svd"]

init_params = ["svd", "random"]

backend_params = ["pytorch", "numpy"]

random_state_param = 42

total_iterations = len(list(product(svd_params, init_params, backend_params))) * len(input_tensors)

#### Implementation

In [45]:
logs = MethodLogger.load_logs_from_file(log_file_path)

for tensor_name, input_tensor in input_tensors.items():
    rank_param = tensor_rank_mapping[tensor_name]

    for backend, svd_func, init_method in tqdm(
        product(backend_params, svd_params, init_params), desc="Проверка набора параметров", total=total_iterations
    ):
        library_method_name = "TensorLy_Tucker"
        method_name = f"{library_method_name}_{tensor_name}_{backend}_{svd_func}_{init_method}"

        if logs:
            existing_log = next(
                (
                    log
                    for log in logs
                    if log["method_name"] == method_name
                    and log["method_args"].get("init") == init_method
                    and log["method_args"].get("svd") == svd_func
                    and log["qualitative_metrics"].get("TensorLy backend") == backend
                ),
                None,
            )
            if existing_log:
                print(f"Пропущена итерация: логи уже существуют для {method_name}")
                continue

        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        gc.collect()

        try:
            with tl.backend_context(backend):
                if backend == "pytorch":
                    tensor_param = tl.tensor(input_tensor).to(torch_device)
                elif backend == "numpy" or backend is None:
                    tensor_param = tl.tensor(input_tensor)

                method_runner = MethodRunner(
                    func=tl.decomposition.tucker,
                    method_input_tensor=tensor_param,
                    library_method_name=library_method_name,
                    backend_name=backend,
                    gpu_memory_tracker=GPUTorchMemoryTracker(),
                    ram_memory_tracker=RAMMemoryTracker(),
                    time_tracker=TimeTracker(),
                )

                method_logger = MethodLogger(
                    method_name=method_name,
                    qualitative_metrics={
                        "Language": "Python",
                        "Library": "TensorLy",
                        "TensorLy backend": f"{backend}",
                        "Tensor type": "Dense",
                        "Data type": tensor_name,
                        "Platform": "CPU, GPU",
                        "Decomposition method": "Tucker",
                    },
                    method_args={
                        "tensor": tensor_param,
                        "rank": rank_param,
                        "n_iter_max": n_iter_max_param,
                        "init": init_method,
                        "svd": svd_func,
                        "random_state": random_state_param,
                    },
                    runner=method_runner,
                )

            reconstructed_tensor = method_runner.reconstructed_tensor

            method_logger.save_logs_to_file(is_test=False)

            reconstructed_frames = []

            if backend == "pytorch":
                reconstructed_tensor = reconstructed_tensor.cpu().numpy()
            reconstructed_frames = [normalize_frames(frame) for frame in reconstructed_tensor]

            save_params_combinations = {
                "video": {"name": method_logger.name, "frames": reconstructed_frames, "fps": original_fps, "frame_size": frame_size}
            }
            save_params = save_params_combinations[tensor_name]

            SaveFramesFactory.get_save_methods(frame_name=tensor_name).save_frames(**save_params)

        except (torch.cuda.OutOfMemoryError, MemoryError) as e:
            print(f"Пропущена итерация из-за недостатка памяти: {backend}, {svd_func}, {init_method}. Ошибка: {e!s}")
            torch.cuda.synchronize()
            torch.cuda.empty_cache()
            gc.collect()
            continue

Файл .cache/method_logs.json не найден.


Проверка набора параметров:   8%|▊         | 1/12 [02:52<31:38, 172.56s/it]

Видео сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_Tucker_video_pytorch_truncated_svd_svd.mp4



Проверка набора параметров:  17%|█▋        | 2/12 [04:23<20:43, 124.35s/it]

Видео сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_Tucker_video_pytorch_truncated_svd_random.mp4


...

Видео сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_Tucker_video_numpy_symeig_svd_random.mp4



Проверка набора параметров:  92%|█████████▏| 11/12 [8:15:31<1:43:47, 6227.89s/it]

Видео сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_Tucker_video_numpy_randomized_svd_svd.mp4



Проверка набора параметров: 100%|██████████| 12/12 [9:28:11<00:00, 2840.99s/it]  

Видео сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_Tucker_video_numpy_randomized_svd_random.mp4


In [46]:
del method_runner, method_logger, tensor_param

torch.cuda.synchronize()
torch.cuda.empty_cache()
gc.collect()

485

### Tensor Train - MPS (tensorly.decomposition.tensor_train)

#### Params

In [47]:
tensor_rank_mapping = {
    "video": [1, 500, 302, 500, 1],
}

input_tensors = {
    "video": video_frames.copy(),
}

svd_params = ["truncated_svd", "symeig_svd", "randomized_svd"]

backend_params = ["pytorch", "numpy"]

total_iterations = len(list(product(backend_params, svd_params))) * len(input_tensors)

#### Implementation

In [49]:
logs = MethodLogger.load_logs_from_file(log_file_path)

for tensor_name, input_tensor in input_tensors.items():
    rank_param = tensor_rank_mapping[tensor_name]
    for backend, svd_func in tqdm(product(backend_params, svd_params), desc="Проверка набора параметров", total=total_iterations):
        library_method_name = "TensorLy_TensorTrain"
        method_name = f"{library_method_name}_{tensor_name}_{backend}_{svd_func}"

        if logs:
            existing_log = next(
                (
                    log
                    for log in logs
                    if log["method_name"] == method_name
                    and log["method_args"].get("svd") == svd_func
                    and log["qualitative_metrics"].get("TensorLy backend") == backend
                ),
                None,
            )
            if existing_log:
                print(f"Пропущена итерация: логи уже существуют для {method_name}")
                continue

        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

        try:
            with tl.backend_context(backend):
                if backend == "pytorch":
                    tensor_param = tl.tensor(input_tensor).to(torch_device)
                elif backend == "numpy" or backend is None:
                    tensor_param = tl.tensor(input_tensor)

                method_runner = MethodRunner(
                    func=tl.decomposition.tensor_train,
                    method_input_tensor=tensor_param,
                    library_method_name=library_method_name,
                    backend_name=backend,
                    gpu_memory_tracker=GPUTorchMemoryTracker(),
                    ram_memory_tracker=RAMMemoryTracker(),
                    time_tracker=TimeTracker(),
                )

                method_logger = MethodLogger(
                    method_name=method_name,
                    qualitative_metrics={
                        "Language": "Python",
                        "Library": "TensorLy",
                        "TensorLy backend": f"{backend}",
                        "Tensor type": "Dense",
                        "Data type": tensor_name,
                        "Platform": "CPU, GPU",
                        "Decomposition method": "TensorTrain",
                    },
                    method_args={
                        "input_tensor": tensor_param,
                        "rank": rank_param,
                        "svd": svd_func,
                    },
                    runner=method_runner,
                )

            reconstructed_tensor_from_tt_factors = method_runner.reconstructed_tensor

            method_logger.save_logs_to_file(is_test=False)

            reconstructed_frames = []

            for tt_factor in reconstructed_tensor_from_tt_factors:
                if backend == "pytorch":
                    reconstructed_frames.append(normalize_frames(tt_factor.cpu().numpy()))
                else:
                    reconstructed_frames.append(normalize_frames(tt_factor))

            save_params_combinations = {
                "video": {"name": method_logger.name, "frames": reconstructed_frames, "fps": original_fps, "frame_size": frame_size}
            }
            save_params = save_params_combinations[tensor_name]

            SaveFramesFactory.get_save_methods(frame_name=tensor_name).save_frames(**save_params)

        except (torch.cuda.OutOfMemoryError, MemoryError) as e:
            print(f"Пропущена итерация из-за недостатка памяти: {backend}, {svd_func}. Ошибка: {e!s}")
            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
            continue

Файл .cache/method_logs.json не найден.


Проверка набора параметров:  17%|█▋        | 1/6 [00:29<02:28, 29.78s/it]

Видео сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_TensorTrain_video_pytorch_truncated_svd.mp4



Проверка набора параметров:  33%|███▎      | 2/6 [00:37<01:06, 16.69s/it]

Пропущена итерация из-за недостатка памяти: pytorch, symeig_svd. Ошибка: CUDA out of memory. Tried to allocate 177.30 GiB (GPU 0; 11.00 GiB total capacity; 2.16 GiB already allocated; 7.36 GiB free; 2.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


...

Видео сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_TensorTrain_video_numpy_truncated_svd.mp4



Проверка набора параметров:  83%|████████▎ | 5/6 [19:05<04:35, 275.89s/it]

Пропущена итерация из-за недостатка памяти: numpy, symeig_svd. Ошибка: Unable to allocate 44.3 GiB for an array with shape (218160, 218160) and data type uint8



Проверка набора параметров: 100%|██████████| 6/6 [2:16:03<00:00, 1360.60s/it]

Видео сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_TensorTrain_video_numpy_randomized_svd.mp4


In [50]:
del method_runner, method_logger, tensor_param

torch.cuda.synchronize()
torch.cuda.empty_cache()
gc.collect()

0

## T3F

### Params

In [51]:
tensor_rank_mapping = {
    "video": [1, 500, 302, 500, 1],
}

input_tensors = {
    "video": video_frames.copy().astype(np.float32),
}

backend_param = "tensorflow"

total_iterations = len(input_tensors)

### Implementation

In [57]:
logs = MethodLogger.load_logs_from_file(log_file_path)

for tensor_name, input_tensor in tqdm(input_tensors.items(), desc="Проверка набора параметров", total=total_iterations):
    rank_param = tensor_rank_mapping[tensor_name]
    tensor_param = input_tensor

    library_method_name = "T3F_TensorTrain"
    method_name = f"{library_method_name}_{tensor_name}"

    if logs:
        existing_log = next((log for log in logs if log["method_name"] == method_name), None)
        if existing_log:
            error_message = f"Пропущена итерация: логи уже существуют для {method_name}"
            raise error_message

    gc.collect()

    method_runner = MethodRunner(
        func=t3f.to_tt_tensor,
        method_input_tensor=tensor_param,
        library_method_name=library_method_name,
        backend_name=backend_param,
        gpu_memory_tracker=GPUTorchMemoryTracker(),
        ram_memory_tracker=RAMMemoryTracker(),
        time_tracker=TimeTracker(),
    )

    method_logger = MethodLogger(
        method_name=method_name,
        qualitative_metrics={
            "Language": "Python",
            "Library": "T3F",
            "T3F backend": f"{backend_param}",
            "Tensor type": "Dense",
            "Data type": tensor_name,
            "Platform": "CPU, GPU",
            "Decomposition method": "TensorTrain",
        },
        method_args={
            "tens": tensor_param,
            "max_tt_rank": rank_param,
        },
        runner=method_runner,
    )

    reconstructed_tensor_t3f = method_runner.reconstructed_tensor

    method_logger.save_logs_to_file(is_test=False)

    reconstructed_frames = []

    for frame in reconstructed_tensor_t3f:
        reconstructed_frames.append(normalize_frames(frame))

    save_params_combinations = {"video": {"name": method_logger.name, "frames": reconstructed_frames, "fps": original_fps, "frame_size": frame_size}}
    save_params = save_params_combinations[tensor_name]

    SaveFramesFactory.get_save_methods(frame_name=tensor_name).save_frames(**save_params)

Файл .cache/method_logs.json не найден.


Проверка набора параметров: 100%|██████████| 1/1 [00:40<00:00, 40.63s/it]

Видео сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/T3F_TensorTrain_video.mp4


In [58]:
del method_runner, method_logger, tensor_param

torch.cuda.synchronize()
torch.cuda.empty_cache()
gc.collect()

0

## TeNPy

### Params

In [25]:
# tensor_param = video_frames.copy().astype(np.float32)
#
# # Размерность физического индекса
# d = tensor_param.shape[-1]  # в вашем случае это 3
#
# # Создаем объект LegCharge
# leg = LegCharge.from_trivial(d)
#
# # Создаем объекты Site для каждого физического индекса, кроме последнего
# sites = [Site(leg) for _ in range(tensor_param.ndim - 1)]
#
# rank_param = [1, 500, 302, 500, 1]

### Implementation

In [26]:
# mps = MPS.from_full(sites=sites, psi=tensor_param, normalize=False)
#
# reconstructed_tensor = mps.to_full_tensor()

AttributeError: 'numpy.ndarray' object has no attribute 'has_label'

In [16]:
# logs = load_logs_from_file(log_file_path)
#
# method_name = f"TeNPy_TensorTrain"
#
# if logs:
#     existing_log = next(
#         (log for log in logs if log['method_name'] == method_name),
#         None
#     )
#     if existing_log:
#         error_message = f"Пропущена итерация: логи уже существуют для {method_name}"
#         raise error_message
#
# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.synchronize()
#
# method_logs = MethodLogger(
#     method_name=method_name,
#     method_input_tensor=tensor_param,
#     qualitative_metrics={
#         "Language": "Python",
#         "Library": "TeNPy",
#         "Tensor type": "Dense",
#         "Platform": "CPU",
#         "Decomposition method": "TensorTrain",
#     },
#     method_args={
#         "sites": tensor_param,
#         "psi": rank_param,
#         "normalize": False,
#     },
#     func=tenpy.networks.mps.MPS.from_full
# )
#
# method_logs_list.append(method_logs)
#
# tt_factors = method_logs.method_result
#
# reconstruct_frames_from_tenpy_tt_factors = tt_factors.to_full_tensor()

Эксперимент набора параметров:   0%|          | 0/1 [00:00<?, ?it/s]


ValueError: unknown type of a

In [ ]:
# method_logs.quantitative_metrics['compression_ratio'] = (100.0 * get_tensors_size(*tt_factors) / get_tensors_size(tensor_param))

# method_logs.quantitative_metrics['frobenius_error'] = (
#         100.0 * np.linalg.norm(reconstruct_frames_from_tenpy_tt_factors - tensor_param) / tl.norm(
#         tensor_param))
#
# save_logs_to_file(method_logs=method_logs, is_test=True)
#
# reconstructed_frames = []
#
# for i in range(len(reconstruct_frames_from_tenpy_tt_factors)):
#     reconstructed_frames.append(normalize_frame_tensorly_tensortrain(reconstruct_frames_from_tenpy_tt_factors[i]))
#
# save_frames_as_video(name=method_logs.name, frames=reconstructed_frames, fps=original_fps, frame_size=frame_size)